In [ ]:
def scrape_imdb():
    import csv
    import re
    from requests import get
    from bs4 import BeautifulSoup
    from IPython.core.display import clear_output
    from warnings import warn
    from time import time, sleep
    from random import gauss, shuffle, randint


    start_time = time()
    html_soup = BeautifulSoup(response.text, 'html.parser')
    page_count = 10

    years = [i for i in range(1935, 2020)]
    year_pages = [(year, page) for year in years for page in range(page_count)]
    # shuffle(year_pages)

    headers = {"Accept-Language": "en-US, en;q=0.5"}

    # Preparing the monitoring of the loop
    start_time = time()
    requests = 0

    # Write the header to file before looping
    with open('output/movie_ratings.csv', 'w', newline='') as output_file:
        csv_headers = ['imdb_ids', 'movie', 'year', 'imdb', 'metascore', 'votes']
        writer = csv.DictWriter(output_file, fieldnames=csv_headers)
        writer.writeheader()

    for year, page in year_pages:

        # Redeclaring the lists to store data in
        names = []
        years = []
        imdb_ratings = []
        metascores = []
        votes = []
        imdb_ids = []

        # Make a get request
        start = str((page * 50) + 1)
        url = 'http://www.imdb.com/search/title?release_date=' + str(year) + '&sort=num_votes,desc&start=' + start 
        while(requests / elapsed_time > 0.1):
            sleep(randint(1, 10))
            elapsed_time = time() - start_time
        response = get(url, headers = headers)

        # Pause the loop
        sleep_time = gauss(10, 4)
        sleep(abs(sleep_time))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s\nLast URL: {}'.format(requests, requests/elapsed_time, url))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            continue

        # Break the loop if the number of requests is greater than expected
        if requests > 2000:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # Scrape the name
            title_html = container.h3.a
            imdb_id = re.search('/title/tt(\d+)/', container.h3.a['href']).group(1)
            imdb_ids.append(imdb_id)
            names.append(title_html.text)

            # Scrape the year
            # year = container.h3.find('span', class_ = 'lister-item-year').text
            years.append(year)

            # Scrape the IMDB rating
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)

            # Scrape the Metascore
            if container.find('div', class_ = 'ratings-metascore') is not None:
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))
            else:
                metascores.append(None)

            # Scrape the number of votes
            vote = container.find('span', attrs = {'name':'nv'})['data-value']
            votes.append(int(vote))

        movie_ratings_df = pd.DataFrame({'imdb_ids': imdb_ids,
                                         'movie': names,
                                         'year': years,
                                         'imdb': imdb_ratings,
                                         'metascore': metascores,
                                         'votes': votes})
        # Checkpoint data as you go
        with open('output/movie_ratings.csv', 'a', newline='') as output_file:
            movie_ratings_df.to_csv(output_file, header=False, index=False)

    sleep(1)
    clear_output(wait = True)

    normalized_movie_ratings_df = pd.read_csv('output/movie_ratings.csv', encoding='ISO-8859-1')
    normalized_movie_ratings_df = normalized_movie_ratings_df[['imdb_ids', 'movie', 'year', 'imdb', 'metascore', 'votes']]
    normalized_movie_ratings_df['imdb'] = normalized_movie_ratings_df['imdb'] * 10
    normalized_movie_ratings_df['imdb'] = normalized_movie_ratings_df['imdb'].astype(int)
    normalized_movie_ratings_df.to_csv('output/norm_movie_ratings.csv')

    display(normalized_movie_ratings_df)
scrape_imdb()

In [2]:
def scrape_thenumbers():
    import bs4
    import requests
    import json
    import os
    import re
    import time
    # There are 5860 movies at the time this program was written
    PAGE_COUNT = 59


    class SimpleLogger(object):
        def __init__(self, filename):
            self.file = open(filename, 'w')

        def __del__(self):
            self.file.close()

        def log(self, msg):
            print(msg)
            print(msg, file=self.file)

    def parse_dollars(dollar_str):
        return int(dollar_str.replace('$', '').replace(',', ''))


    def clean_input(input_str):
        return input_str.encode("ascii", errors="ignore").decode()


    def scrape_movie(url_ext):
        page_str = 'https://www.the-numbers.com' + url_ext
        movie_rsp = requests.get(page_str)
        
        try:
            movie_rsp.raise_for_status()
        except requests.HTTPError:
            logger.log('Request for ' + page_str + ' failed!')
            return None
        parsed_movie_rsp = bs4.BeautifulSoup(clean_input(movie_rsp.text), features='html.parser')
        table = parsed_movie_rsp.find(text='Movie Details').parent.find_next('table')
        mpaa_rating = None
        running_time = None
        franchises = None
        keywords = None
        source = None
        genre = None
        production_method = None
        creative_type = None
        production_companies = None
        production_countries = None
        languages = None
        temp = table.find(text='MPAA Rating:')
        if temp:
            result = re.search('/market/mpaa-rating/([\w\d\(\)-]+)', temp.parent.parent.parent.select('a')[0]['href'])
            mpaa_rating = {
                'slug': result.group(1) if result else None,
                'pretty': temp.parent.parent.parent.select('a')[0].text
            }
        temp = table.find(text='Running Time:')
        if temp:
            temp = temp.parent.parent.find_next('td').getText()
            running_time = int(temp.split(' ')[0])
        temp = table.find(text='Franchise:')
        if temp:
            franchises = list()
            for franchise in temp.parent.find_next('td').find_all('a'):
                result = re.search('/movies/franchise/([\w\d\(\)-]+)', franchise['href'])
                franchises.append({
                    'slug': result.group(1) if result else None,
                    'pretty': franchise.text
                })
        temp = table.find(text='Keywords:')
        if temp:
            keywords = list()
            for keyword in temp.parent.find_next('td').find_all('a'):
                result = re.search('/movies/keywords/([\w\d\(\)-]+)', keyword['href'])
                keywords.append({
                    'slug': result.group(1) if result else None,
                    'pretty': keyword.text
                })
        temp = table.find(text='Source:')
        if temp:
            result = re.search('/market/source/([\w\d\(\)-]+)', temp.parent.parent.parent.select('a')[0]['href'])
            source = {
                'slug': result.group(1) if result else None,
                'pretty': temp.parent.parent.parent.select('a')[0].getText()
            }
        temp = table.find(text='Genre:')
        if temp:
            result = re.search('/market/genre/([\w\d\(\)-]+)', temp.parent.parent.parent.select('a')[0]['href'])
            genre = {
                'slug': result.group(1) if result else None,
                'pretty': temp.parent.parent.parent.select('a')[0].getText()
            }
        temp = table.find(text='Production Method:')
        if temp:
            result = re.search('/market/production-method/([\w\d\(\)-]+)', temp.parent.parent.parent.select('a')[0]['href']) 
            production_method = {
                'slug': result.group(1) if result else None,
                'pretty': temp.parent.parent.parent.select('a')[0].getText()
            }
        temp = table.find(text='Creative Type:')
        
        print(table)
        print(temp)
        if temp:
            result = re.search('/market/creative-type/([\w\d\(\)-]+)', temp.parent.parent.parent.select('a')[0]['href'])
            creative_type = {
                'slug': result.group(1) if result else None,
                'pretty': temp.parent.parent.parent.select('a')[0].getText()
            }
        temp = table.find(text='Production Companies:')
        if temp:
            production_companies = list()
            for production_company in temp.parent.find_next('td').find_all('a'):
                result = re.search('/movies/production-company/([\w\d\(\)-]+)', production_company['href'])
                production_companies.append({
                    'slug': result.group(1) if result else None,
                    'pretty': production_company.text
                })
        temp = table.find(text='Production Countries:')
        if temp:
            production_countries = list()
            for production_country in temp.parent.find_next('td').find_all('a'):
                result = re.search('/([\w\d\(\)-]+)/movies', production_country['href'])
                production_countries.append({
                    'slug': result.group(1) if result else None, 
                    'pretty': production_country.text
                })
        temp = table.find(text='Languages:')
        if temp:
            languages = list()
            for language in temp.parent.find_next('td').find_all('a'):
                result = re.search('/language/([\w\d\(\)-]+)/movies', language['href'])
                languages.append({
                    'slug': result.group(1) if result else None,
                    'pretty': language.text
                })
        movie_dict = {
            'mpaa-rating': mpaa_rating,
            'running-time': running_time,
            'franchises': franchises,
            'keywords': keywords,
            'source': source,
            'genre': genre,
            'production_method': production_method,
            'creative_type': creative_type,
            'production_companies': production_companies,
            'production_countries': production_countries,
            'languages': languages
        }
        return movie_dict['creative_type']

    print(scrape_movie('/movie/Star-Wars-Ep-VIII-The-Last-Jedi'))
    return

    def scrape_list_page(url_ext):
        count = 0
        page_str = 'https://www.the-numbers.com/movie/budgets/all' + url_ext
        movie_lst_rsp = requests.get(page_str)
        # Halt if there was an issue with the request
        try:
            movie_lst_rsp.raise_for_status()
        except requests.HTTPError:
            logger.log('Request for ' + page_str + ' failed!')
            return None
        else:
            logger.log('Request for ' + page_str + ' succeeded')
        parsed_movie_lst_rsp = bs4.BeautifulSoup(clean_input(movie_lst_rsp.text), features="html.parser")
        for table_row in parsed_movie_lst_rsp.find_all('tr')[1:]:
            table_cells = table_row.find_all('td')
            i = 0
            movie_dict = dict()
            result = re.search('/box-office-chart/daily/(\d{4}/\d{1,2}/\d{1,2})', table_cells[i + 1].a['href'])
            movie_dict['release_date'] = {
                'slug': result.group(1) if result else None,
                'pretty': table_cells[i + 1].a.text
            }
            movie_dict['title'] = {
                'slug': re.search('/movie/([\w\d\(\)-]+)#tab=summary', table_cells[i + 2].b.a['href']).group(1),
                'pretty': table_cells[i + 2].b.a.text
            }
            # Scrape the movie page
            movie_dict['production_budget'] = parse_dollars(table_cells[i + 3].getText())
            movie_dict['domestic_gross'] = parse_dollars(table_cells[i + 4].getText())
            movie_dict['worldwide_gross'] = parse_dollars(table_cells[i + 5].getText())
            movie_dict.update(scrape_movie(table_cells[i + 2].select('a')[0]['href']))
            yield movie_dict
            count += 1
        logger.log('Read ' + str(count) + ' entries from page ' + page_str)


    def scrape_the_numbers_generator():
        url_ext_lst = [''] + ['/' + str(x) + '01' for x in range(1, PAGE_COUNT)]
        for url_ext in url_ext_lst:
            for page in scrape_list_page(url_ext):
                if page is None:
                    logger.log('Page {} returned None'.format(url_ext))
                    break
                else:
                    yield page


    def store_the_numbers(filename):
        start_total = time.time()
        i = 1
        start_row = time.time()
        entries = list()
        for entry in scrape_the_numbers_generator():
            entries.append(entry)
            exec_time_row = time.time() - start_row
            logger.log('In {:.3f} seconds - {:0>4} {}'.format(exec_time_row, i, entry['title']['pretty']))
            start_row = time.time()
            i += 1
            if i % 100 == 0:
                with open(filename, 'w', newline='') as outfile:
                    json.dump(entries, outfile)
        with open(filename, 'w', newline='') as outfile:
                    json.dump(entries, outfile)
        exec_time_total = time.time() - start_total
        logger.log('{} total entries in {:.3f} seconds'.format(i - 1, exec_time_total))


    if __name__ == '__main__':
        filename = 'output/the_numbers.json'        
        # We really don't want to overwrite any existing good DB file, it takes a long time to scrape all of the data
        if os.path.isfile(filename):
            exit()
        logger = SimpleLogger('log.txt')
        store_the_numbers(filename)

scrape_thenumbers()

<table>
<tr><td><b>Production Budget:</b></td><td>$317,000,000</td></tr>
<meta content="2017-12-13" itemprop="dateCreated"/>
<tr><td><b>Domestic Releases:</b></td>
<td>December 15th, 2017 (Wide) by <a href="/market/distributor/Walt-Disney">Walt Disney</a><br/>December 15th, 2017 (IMAX) by <a href="/market/distributor/Walt-Disney">Walt Disney</a></td></tr>
<tr><td><b>International Releases:</b></td>
<td>December 13th, 2017 (Wide) (<a href="/movie/Star-Wars-Ep-VIII-The-Last-Jedi/Belgium#tab=summary" rel="nofollow">Belgium</a>)
<br/>December 13th, 2017 (Wide) (<a href="/movie/Star-Wars-Ep-VIII-The-Last-Jedi/Brazil#tab=summary">Brazil</a>)
<br/>December 13th, 2017 (Wide) (<a href="/movie/Star-Wars-Ep-VIII-The-Last-Jedi/Colombia#tab=summary" rel="nofollow">Colombia</a>)
<br/>December 13th, 2017 (Wide) (<a href="/movie/Star-Wars-Ep-VIII-The-Last-Jedi/Denmark#tab=summary">Denmark</a>)
<br/>December 13th, 2017 (Wide) (<a href="/movie/Star-Wars-Ep-VIII-The-Last-Jedi/Finland#tab=summary" rel="no

In [91]:
def normalize_movies():
    import json
    import re
    import time
    import tmdbsimple as tmdb
    from requests import HTTPError
    import sys
    
    tmdb.API_KEY = 'af3f6fb737c9085bd424c403dc83c196'
    
    def simple_test():
        search = tmdb.Search()
        results = search.movie(**{'query': 'Star Wars Episode VIII The Last Jedi', 
                                  'year': '2017'})
        movie = tmdb.Movies(results['results'][0]['id'])
        results = movie.external_ids()
        print(movie.external_ids()['imdb_id'])
        sys.exit()
    # simple_test()
    
    class SimpleLogger(object):
        def __init__(self, filename):
            self.file = open(filename, 'w')

        def __del__(self):
            self.file.close()

        def log(self, msg):
            print(msg)
            print(msg, file=self.file)
            
    def clean_string(in_str):
        return re.sub('[^A-Za-z0-9 \-]+', '', in_str)
    
    def normalize_movie_title(in_str):
        in_str = clean_string(in_str)
        in_str = re.sub(' ([Ee]p) ', ' \g<1>isode ', in_str)
        return in_str
            
    def search_imdb_id(title, year):
        # print('\"{}\" ({})'.format(title, year))
        search = tmdb.Search()
        query_dict = {'query': title}
        if year is not None:
            query_dict['year'] = str(year)
        results = search.movie(**query_dict)
        if (not 'results' in results) or (not results['results']) or (not 'id' in results['results'][0]):
            return None
        movie = tmdb.Movies(results['results'][0]['id'])
        results = movie.external_ids()
        if not 'imdb_id' in results or results['imdb_id'] is  None:
            return None
        result = re.search('tt(\d+)', results['imdb_id'])
        return result.group(1) if result else None
    
    
    def normalize_the_numbers():
        with open('data/the_numbers/the_numbers.json', 'r') as input_file:
            the_numbers = json.load(input_file)
        for i, movie in enumerate(the_numbers):
            year = None
            if movie['release_date']['slug'] is not None:
                result = re.search('^(\d{4}).*', movie['release_date']['slug'])
                year = int(result.group(1)) if result is not None else None
                if 1900 < year < 2030:
                    year = str(year)
                else:
                    year = None
            try:
                imdb_id = search_imdb_id(normalize_movie_title(movie['title']['slug']), year)
                if imdb_id is None:
                    time.sleep(0.26)
                    imdb_id = search_imdb_id(normalize_movie_title(movie['title']['pretty']), year)
                the_numbers[i]['imdb_id'] = imdb_id
            except HTTPError:
                print('HTTPError: {} ({})'.format(title, year))
            # API Limited to 40 calls every 10 seconds, search_imdb_id calls API twice for each movie
            logger.log('{:0>4} tt{: <10} {} ({})'.format(i, imdb_id if imdb_id is not None else '', movie['title']['pretty'], year))
            time.sleep(0.51)
        with open('output/the_numbers_norm.json', 'w', newline='') as outfile:
            json.dump(the_numbers, outfile)
    
    logger = SimpleLogger('normalize_log.txt')
    normalize_the_numbers()

normalize_movies()

0000 tt0499549    Avatar (2009)
0001 tt1298650    Pirates of the Caribbean: On Stranger Tides (2011)
0002 tt4154796    Avengers: Endgame (2019)
0003 tt2395427    Avengers: Age of Ultron (2015)
0004 tt2527336    Star Wars Ep. VIII: The Last Jedi (2017)
0005 tt2488496    Star Wars Ep. VII: The Force Awakens (2015)
0006 tt4154756    Avengers: Infinity War (2018)
0007 tt0449088    Pirates of the Caribbean: At Worlds End (2007)
0008 tt0974015    Justice League (2017)
0009 tt2379713    Spectre (2015)
0010 tt1345836    The Dark Knight Rises (2012)
0011 tt3778644    Solo: A Star Wars Story (2018)
0012 tt1210819    The Lone Ranger (2013)
0013 tt0401729    John Carter (2012)
0014 tt0398286    Tangled (2010)
0015 tt0413300    Spider-Man 3 (2007)
0016 tt3498820    Captain America: Civil War (2016)
0017 tt2975590    Batman v Superman: Dawn of Justice (2016)
0018 tt0903624    The Hobbit: An Unexpected Journey (2012)
0019 tt0417741    Harry Potter and the Half-Blood Prince (2009)
0020 tt1170358    Th

0181 tt0242653    The Matrix Revolutions (2003)
0182 tt1677720    Ready Player One (2018)
0183 tt0371606    Chicken Little (2005)
0184 tt0317919    Mission: Impossible III (2006)
0185 tt0808151    Angels & Demons (2009)
0186 tt0332452    Troy (2004)
0187 tt0938283    The Last Airbender (2010)
0188 tt0389790    Bee Movie (2007)
0189 tt0436339    G-Force (2009)
0190 tt0397892    Bolt (2008)
0191 tt1646987    Wrath of the Titans (2012)
0192 tt0442933    Beowulf (2007)
0193 tt1077368    Dark Shadows (2012)
0194 tt2334879    White House Down (2013)
0195 tt0780653    The Wolfman (2010)
0196 tt2034800    The Great Wall (2016)
0197 tt3332064    Pan (2015)
0198 tt1305591    Mars Needs Moms (2011)
0199 tt5834426    Moonfall (None)
0200 tt0424095    Flushed Away (2006)
0201 tt1277953    Madagascar 3: Europe's Most Wanted (2012)
0202 tt1229238    Mission: ImpossibleGhost Protocol (2011)
0203 tt1646971    How to Train Your Dragon 2 (2014)
0204 tt0120855    Tarzan (1999)
0205 tt0864835    Mr. Peabod

0370 tt0112462    Batman Forever (1995)
0371 tt0133152    Planet of the Apes (2001)
0372 tt0368891    National Treasure (2004)
0373 tt1853728    Django Unchained (2012)
0374 tt0970866    Little Fockers (2010)
0375 tt0111503    True Lies (1994)
0376 tt0765429    American Gangster (2007)
0377 tt0844471    Cloudy with a Chance of Meatballs (2009)
0378 tt1386588    The Other Guys (2010)
0379 tt0339291    Lemony Snicket's A Series of Unfortunate Events (2004)
0380 tt0993846    The Wolf of Wall Street (2013)
0381 tt0848537    Epic (2013)
0382 tt0116213    Eraser (1996)
0383 tt0116583    The Hunchback of Notre Dame (1996)
0384 tt0120917    The Emperor's New Groove (2000)
0385 tt1320253    The Expendables 2 (2012)
0386 tt0386117    Where the Wild Things Are (2009)
0387 tt0449010    Eragon (2006)
0388 tt1267297    Hercules (2014)
0389 tt1243957    The Tourist (2010)
0390 tt0146675    End of Days (1999)
0391 tt0327162    The Stepford Wives (2004)
0392 tt0423294    Surf's Up (2007)
0393 tt0450259

0561 tt0337741    Something's Gotta Give (2003)
0562 tt0369436    Four Christmases (2008)
0563 tt0119094    Face/Off (1997)
0564 tt0960731    Bedtime Stories (2008)
0565 tt0257044    Road to Perdition (2002)
0566 tt1564367    Just Go With It (2011)
0567 tt0287978    Daredevil (2003)
0568 tt1059786    Eagle Eye (2008)
0569 tt0118880    Con Air (1997)
0570 tt1302067    Yogi Bear (2010)
0571 tt0159365    Cold Mountain (2003)
0572 tt1037705    The Book of Eli (2010)
0573 tt0119137    Flubber (1997)
0574 tt0171363    The Haunting (1999)
0575 tt0117705    Space Jam (1996)
0576 tt2713180    Fury (2014)
0577 tt6182908    Smallfoot (2018)
0578 tt0383216    The Pink Panther (2006)
0579 tt1222817    Zookeeper (2011)
0580 tt0043456    The Day the Earth Stood Still (2008)
0581 tt0118883    Conspiracy Theory (1997)
0582 tt0120828    Six Days, Seven Nights (1998)
0583 tt0166813    Spirit: Stallion of the Cimarron (2002)
0584 tt0244244    Swordfish (2001)
0585 tt           Fantasia 2000 (Theatrical Re

0764 tt1325004    The Twilight Saga: Eclipse (2010)
0765 tt0164184    The Sum of All Fears (2002)
0766 tt2637276    Ted 2 (2015)
0767 tt0227445    The Score (2001)
0768 tt0480687    Hall Pass (2011)
0769 tt0113845    Money Train (1995)
0770 tt0285531    Dreamcatcher (2003)
0771 tt0132347    Mystery Men (1999)
0772 tt0140352    The Insider (1999)
0773 tt1389072    Downsizing (2017)
0774 tt1482459    Doctor Seuss' The Lorax (2012)
0775 tt0758774    Body of Lies (2008)
0776 tt1611224    Abraham Lincoln: Vampire Hunter (2012)
0777 tt0116830    Last Man Standing (1996)
0778 tt0462396    The Last Legion (2007)
0779 tt0137494    Entrapment (1999)
0780 tt0365474    The X Files: Fight the Future (1998)
0781 tt2369135    Need for Speed (2014)
0782 tt0120815    Saving Private Ryan (1998)
0783 tt0207201    What Women Want (2000)
0784 tt0443272    Lincoln (2012)
0785 tt0268380    Ice Age (2002)
0786 tt0112384    Apollo 13 (1995)
0787 tt0133093    The Matrix (1999)
0788 tt5220122    Hotel Transylvan

...

4977 tt0379557    Touching the Void (2004)
4978 tt1645089    Inside Job (2010)
4979 tt0029852    Alexander's Ragtime Band (1938)
4980 tt0415978    Me and You and Everyone We Know (2005)
4981 tt3787590    We Are Your Friends (2015)
4982 tt0433387    Harsh Times (2006)
4983 tt0165798    Ghost Dog: The Way of the Samurai (2000)
4984 tt2326612    Captive (2015)
4985 tt0290212    Full Frontal (2002)
4986 tt7158430    Hearts Beat Loud (2018)
4987 tt5816564    The Resurrection of Gavin Stone (2017)
4988 tt1185616    Waltz with Bashir (2008)
4989 tt0369994    Strangers with Candy (2006)
4990 tt0102536    Night on Earth (1992)
4991 tt0107668    Nemesis (1993)
4992 tt0367027    Shortbus (2006)
4993 tt           Son of Rambow: A Home Movie (2008)
4994 tt0349159    The Book of Mormon Movie, Volume 1: The Journey (2003)
4995 tt0171359    Hamlet (2000)
4996 tt3172532    The Diary of a Teenage Girl (2015)
4997 tt0912593    No End In Sight (2007)
4998 tt2692904    Locke (2014)
4999 tt0162973    Get Re

5176 tt           Mutant World (2015)
5177 tt3526286    #Horror (2015)
5178 tt3781616    Checkmate (2015)
5179 tt0029583    Snow White and the Seven Dwarfs (1937)
5180 tt0063592    Shalako (1968)
5181 tt4285496    El abrazo de la serpiente (2016)
5182 tt5311542    Werk ohne Autor (2018)
5183 tt3966404    Mustang (2015)
5184 tt0300270    The Holy Girl (2005)
5185 tt3230082    La chambre bleue (2014)
5186 tt0374639    Incident at Loch Ness (2004)
5187 tt1480658    Eddie: The Sleepwalking Cannibal (2013)
5188 tt1447793    My Girlfriend's Boyfriend (2010)
5189 tt2640460    Kurmanjan datka (2014)
5190 tt0464054    House at the End of the Drive (2014)
5191 tt0060153    Batman - The Movie (1966)
5192 tt0120735    Lock, Stock and Two Smoking Barrels (1999)
5193 tt3551840    The Ballad of Gregorio Cortez (1983)
5194 tt0028216    San Francisco (1936)
5195 tt0154420    Festen (1998)
5196 tt0117958    Trees Lounge (1996)
5197 tt1432078    The Girlfriend Experience (2009)
5198 tt0433398    Journey 

5372 tt           The Poker House (2009)
5373 tt0448090    Proud (2005)
5374 tt           Steppin: The Movie (2008)
5375 tt1134674    Zombies of Mass Destruction (2010)
5376 tt2149137    Snow White: A Deadly Summer (2012)
5377 tt1838722    Truth or Die (2012)
5378 tt1930294    Black Rock (2013)
5379 tt3289362    Hidden Away (2014)
5380 tt           My Last Day Without You (2013)
5381 tt2446502    Zombie Hunter (2013)
5382 tt3359872    Doc Holliday's Revenge (2014)
5383 tt1604100    A Fine Step (2014)
5384 tt2165765    Fear Clinic (2015)
5385 tt4676372    The Pet (2015)
5386 tt2655734    Bang Bang Baby (2014)
5387 tt5022702    Hush (2016)
5388 tt           1982 (2016)
5389 tt3034146    The Horror Network (2015)
5390 tt3406296    Chemical Cut (2016)
5391 tt           Por amor en el caserio (2015)
5392 tt8361028    Cam (2018)
5393 tt8327962    Alien Psychosis (2018)
5394 tt0825346    And Then Came Love (2007)
5395 tt0424136    Hard Candy (2006)
5396 tt0330136    Charly (2002)
5397 tt20112

5570 tt2112209    Mutual Friends (2014)
5571 tt4273494    Rise Of the Entrepreneur - The Search For A Bet… (2014)
5572 tt0024034    42nd Street (1933)
5573 tt0100142    Metropolitan (1990)
5574 tt0456149    Moartea domnului Lazarescu (2006)
5575 tt           Gory Gory Hallelujah (2005)
5576 tt3843282    Twenty-Two (2017)
5577 tt2359024    Blue Ruin (2014)
5578 tt0374900    Napoleon Dynamite (2004)
5579 tt0071853    Monty Python and the Holy Grail (1975)
5580 tt0451176    Quinceanera (2006)
5581 tt           Dogtown and Z-Boys (2002)
5582 tt1259998    Heroes (2008)
5583 tt           Tarnation (2004)
5584 tt1560957    I Want Your Money (2010)
5585 tt0082307    E tu vivrai nel terrore - L'aldil (1983)
5586 tt0079302    Home Movies (1980)
5587 tt0261755    Jackpot (2001)
5588 tt           Fabled (2004)
5589 tt0402249    The Dark Hours (2005)
5590 tt           Viskningar och rop (1973)
5591 tt0091578    My Beautiful Laundrette (1986)
5592 tt           Show Me (2005)
5593 tt3867652    Pancak

5767 tt3109200    Dude, Where's My Dog (2014)
5768 tt2343473    Echo Dr. (2014)
5769 tt           Closure (2015)
5770 tt           Lunchtime Heroes (2015)
5771 tt3677412    Open Secret (2015)
5772 tt2797642    The Night Visitor (2015)
5773 tt           Tiger Orange (2015)
5774 tt0068833    The Last House on the Left (1972)
5775 tt3470600    Sing (None)
5776 tt           The Foot Fist Way (2008)
5777 tt           Dawn of the Crescent Moon (2015)
5778 tt2319456    Queen Crab (2015)
5779 tt2955096    Happy Christmas (2014)
5780 tt0428959    Peace, Propaganda and the Promised Land (2005)
5781 tt0431830    Absentia (2013)
5782 tt0138704    Pi (1998)
5783 tt           I Love You  Don't Touch Me! (1998)
5784 tt0138987    20 Dates (1999)
5785 tt0390521    Super Size Me (2004)
5786 tt           Supporting Characters (2013)
5787 tt           The FP (2012)
5788 tt3672742    Turbo Kid (2015)
5789 tt1861343    Hayride (2012)
5790 tt0052646    The Brain That Wouldn't Die (1962)
5791 tt2334896    The

In [108]:
def rectify_the_numbers():
    import csv
    import json
    import os
    import pandas as pd
    import re
    
    def rectify_single(the_numbers, output_dir):
        single_columns = \
        ['release_date', 'release_date_norm', 'title', 'title_norm', 'mpaa-rating', 
         'mpaa-rating_norm', 'running-time', 'source', 'source_norm', 'genre', 
         'genre_norm', 'production_method', 'production_method_norm', 
         'creative_type', 'creative_type_norm', 'production_budget', 'domestic_gross', 
         'worldwide_gross', 'imdb_id']
        
        with open(os.path.join(output_dir, 'the_numbers_rectified.csv'), 'w', newline='') as outfile:
            header_writer = csv.writer(outfile)
            header_writer.writerow(columns)
        i = 0
        for entry in the_numbers:
            rectified_entry = dict()
            for key, value in entry.items():
                if key is None:
                    continue
                if type(value) == dict and 'slug' in value and 'pretty' in value:
                    rectified_entry[key]  = value['pretty']
                    rectified_entry[key + '_norm'] = value['slug']
                else:
                    rectified_entry[key] = value
            with open('output/the_numbers_rectified.csv', 'a', newline='') as outfile:
                row_writer = csv.DictWriter(outfile, columns)
                row_writer.writerow(rectified_entry)
            if rectified_entry['release_date_norm'] is not None:
                result = re.search('^(\d{4}).*', rectified_entry['release_date_norm'])
                year = result.group(1) if result is not None else None
            log = '{:0>4} {}'.format(i, rectified_entry['title'])
            if year is not None:
                log += ' ({})'.format(year)
            print(log)
            i += 1
    
    def rectify_lst(df, imdb_id, lst):
        return pd.concat([pd.DataFrame([imdb_id, el], columns=['imdb']) for el in lst])
    
    def rectify_lsts(the_numbers, output_dir):
        lst_columns = ['franchises', 'keywords', 'production_companies', 'production_countries', 'languages']
        pretty_columns = [(col, 'pretty', col) for col in lst_columns]
        slug_columns = [(col, 'slug', col + '_norm') for col in lst_columns]
        all_columns = pretty_columns + slug_columns
        dataframe_dict = {dest_col: pd.DataFrame([], columns=['imdb_id', dest_col]) 
                          for source_col, specifier, dest_col in all_columns}
        for entry in the_numbers:
            if 'imdb_id' not in entry:
                continue
            imdb_id = entry['imdb_id'] 
            for source_col, specifier, dest_col in all_columns:
                if source_col in entry:
                    # print(source_col, ' ', specifier, ' ', dest_col)
                    # dataframe_dict[dest_col] = dataframe_dict[dest_col].append(pd.concat([pd.DataFrame([imdb_id, el], columns=dataframe_dict[dest_col].columns) for el in entry[source_col][specifier]]))
                    dataframe_dict[dest_col].append(pd.concat([pd.DataFrame([[imdb_id, el[specifier]]], columns=dataframe_dict[dest_col].columns) for el in entry[source_col]]))
        display(dataframe_dict['franchises'])
        
    def rectify():
        with open('output/the_numbers_norm.json', 'r') as infile:
            the_numbers = json.load(infile)
        output_dir = os.path.join('output', 'rectified')
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        # rectify_single(the_numbers, output_dir)
        rectify_lsts(the_numbers, output_dir)
        
    rectify()
    
rectify_the_numbers()

TypeError: 'NoneType' object is not iterable

In [94]:
def merge_data():
    import pandas as pd
    
    df_movie_ratings = pd.read_csv('output/norm_movie_ratings.csv', encoding = "ISO-8859-1")
    df_the_numbers = pd.read_csv('output/the_numbers_rectified.csv', encoding = "ISO-8859-1")
    display(df_movie_ratings.head())
    display(df_the_numbers.head())
    df_merged = pd.merge(df_movie_ratings, df_the_numbers, left_on='imdb_ids', right_on='imdb_id')
    df_merged.to_csv('output/merged.csv', index=False)
    
merge_data()

,Unnamed: 0,imdb_ids,movie,year,imdb,metascore,votes
0,0,26029,The 39 Steps,1935,77,NaN,47653
1,1,26138,Bride of Frankenstein,1935,78,NaN,40030
2,2,26778,A Night at the Opera,1935,79,NaN,28959
3,3,26752,Mutiny on the Bounty,1935,77,87.0,19293
4,4,27125,Top Hat,1935,78,NaN,16019


,release_date,release_date_norm,title,title_norm,mpaa-rating,mpaa-rating_norm,running-time,franchises,franchises_norm,keywords,...,production_companies,production_companies_norm,production_countries,production_countries_norm,languages,languages_norm,production_budget,domestic_gross,worldwide_gross,imdb_id
0,"Dec 17, 2009",2009/12/17,Avatar,Avatar,PG-13,PG-13-(US),162.0,Avatar,Avatar,3-D;Invented Language;Visual Effects;Alien Inv...,...,Dune Entertainment;20th Century Fox;Ingenious ...,Dune-Entertainment;20th-Century-Fox;Ingenious-...,United States,United-States,English;Na'vi,English;Navi,425000000,760507625,2789705275,499549.0
1,"May 20, 2011",2011/05/20,Pirates of the Caribbean: On Stranger Tides,Pirates-of-the-Caribbean-On-Stranger-Tides,PG-13,PG-13-(US),136.0,Pirates of the Caribbean,Pirates-of-the-Caribbean,Pirates;Zombies;Visual Effects;Romance;Epilogu...,...,Walt Disney Pictures,Walt-Disney-Pictures,United States,United-States,English,English,410600000,241063875,1045663875,1298650.0
2,"Apr 23, 2019",2019/04/23,Avengers: Endgame,Avengers-Endgame-(2019),PG-13,PG-13-(US),181.0,Marvel Cinematic Universe;Avengers,Marvel-Cinematic-Universe;Avengers,Ensemble;Marvel Comics;Animal Lead;Non-Chronol...,...,Marvel Studios,Marvel-Studios,United States,United-States,English,English,400000000,858373000,2795473000,4154796.0
3,"Apr 22, 2015",2015/04/22,Avengers: Age of Ultron,Avengers-Age-of-Ultron,PG-13,PG-13-(US),141.0,Marvel Cinematic Universe;Avengers,Marvel-Cinematic-Universe;Avengers,Marvel Comics;Robot;End of the World;Hallucina...,...,Marvel Studios,Marvel-Studios,United States,United-States,English,English,330600000,459005868,1403013963,2395427.0
4,"Dec 13, 2017",2017/12/13,Star Wars Ep. VIII: The Last Jedi,Star-Wars-Ep-VIII-The-Last-Jedi,PG-13,PG-13-(US),150.0,Star Wars,Star-Wars,Space Opera;Robot;Good vs. Evil;Outer Space;Dy...,...,Lucasfilm;Walt Disney Pictures,Lucasfilm;Walt-Disney-Pictures,United States,United-States,English,English,317000000,620181382,1316721747,2527336.0
